### **Exercise: Sentiment Analysis and Key Insights Extraction from Ford Car Reviews**

### **Problem Statement:**
You have been provided with a dataset containing Ford car reviews. Your task is to use LangChain and the concepts you’ve learned to perform the following tasks:

1. **Sentiment Analysis**: Analyze the sentiment of each review, categorize it as positive, neutral, or negative, and store the result.
2. **Key Insights Extraction**: Extract key pieces of information from each review, such as the pros and cons mentioned, and the specific features the reviewer liked or disliked (e.g., vehicle performance, comfort, price).

You will build a LangChain-based solution that leverages language models to automatically extract this information and provide a structured summary of the reviews. 

---
### **Steps to Solve:**

#### **Step 1: Load the Dataset**
- The dataset file is named `ford_car_reviews.csv` and is sourced from Kaggle: [Edmunds Consumer Car Ratings and Reviews](https://www.kaggle.com/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews).
- For this exercise, **limit the data to the first 25 records**. This can be achieved by using `df.head(25)` or `df.iloc[:25]` when loading the data into a DataFrame.

#### **Step 2: Define the Sentiment Analysis Task**
- Use LangChain to create a pipeline to classify the sentiment of each review.
- Define prompts that can guide the model to evaluate the sentiment. For example:
  - "Given the following car review, classify the sentiment as positive, neutral, or negative."

#### **Step 3: Key Insights Extraction**
- Use LangChain to create a pipeline to extract pros, cons, and notable features from each review. Define prompts such as:
  - "What are the pros and cons of the vehicle described in the following review?"
  - "What specific features of the vehicle does the reviewer like or dislike?"

#### **Step 4: Update the DataFrame with New Information**
- Run the pipeline for each review and collect the sentiment and insights.
- Once the analysis and extraction are complete, update the original DataFrame with additional columns to include:
  - Sentiment (positive, neutral, negative)
  - Pros
  - Cons
  - Liked_Features
  - Disliked_Features

---

### **Example Output:**

```json
{
  "Review_Date": "03/07/13",
  "Vehicle_Title": "2006 Ford Mustang Coupe",
  "Review_Text": "With the expected arrival of our 6th child...",
  "Rating": 4.125,
  "Sentiment": "Positive",
  "Pros": "Good driving experience, Large seating capacity, Great options",
  "Cons": "None mentioned",
  "Liked_Features": ["Driving experience", "Seating capacity", "Options available"],
  "Disliked_Features": []
}
```

In [1]:
import pandas as pd
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI
from transformers import pipeline
from langchain_groq import ChatGroq

In [2]:
df = pd.read_csv("ford_car_reviews.csv", nrows=25)

In [3]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [4]:
df.head()

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,on 06/06/18 14:19 PM (PDT),Vicki,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,2006 Mustang GT,Doesn’t disappoint,5.0
1,on 08/12/17 06:06 AM (PDT),Tom,2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...,DREAM CAR,I bought mine 4/17 with 98K. Have been wantin...,3.0
2,on 06/15/17 05:43 AM (PDT),Ray,2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...,Great Ride,There will always be a 05-09 mustang for sale...,5.0
3,on 05/18/17 17:33 PM (PDT),Don Watson,2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...,I have wanted a Mustang for 40 years.,I bought my car from an auction I work at ( A...,5.0
4,on 01/03/16 18:03 PM (PST),One owner,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,One owner,I bought this car spankin new and i still am ...,5.0


In [5]:
# Initialize the Groq LLM
llm = ChatGroq(groq_api_key="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") # Replace with your actual API key


In [6]:
# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["review"],
    template="Given the following car review, classify the sentiment as positive, neutral, or negative. Just mention positive, negative or neutral.Nothing extra. \n\nReview: {review}"
)

In [7]:
# Create an LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)


C:\Users\shikh\AppData\Local\Temp\ipykernel_21168\2419055736.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


In [8]:
# Function to classify the sentiment of a single review
def classify_sentiment(review):
    try:
        sentiment = chain.run(review)
        return sentiment
    except Exception as e:
        print(f"Error classifying sentiment for review: {review}. Error: {e}")
        return "Error"


In [9]:
# Apply the sentiment classification function to the 'Review' column
df['Sentiment'] = df['Review'].apply(classify_sentiment)


C:\Users\shikh\AppData\Local\Temp\ipykernel_21168\3420049455.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  sentiment = chain.run(review)


In [10]:
# Print the DataFrame with the added 'Sentiment' column
df.head()

,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,Sentiment
0,on 06/06/18 14:19 PM (PDT),Vicki,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,2006 Mustang GT,Doesn’t disappoint,5.0,Positive
1,on 08/12/17 06:06 AM (PDT),Tom,2006 Ford Mustang Coupe V6 Standard 2dr Coupe ...,DREAM CAR,I bought mine 4/17 with 98K. Have been wantin...,3.0,Negative
2,on 06/15/17 05:43 AM (PDT),Ray,2006 Ford Mustang Coupe V6 Premium 2dr Coupe (...,Great Ride,There will always be a 05-09 mustang for sale...,5.0,Positive
3,on 05/18/17 17:33 PM (PDT),Don Watson,2006 Ford Mustang Coupe V6 Deluxe 2dr Coupe (4...,I have wanted a Mustang for 40 years.,I bought my car from an auction I work at ( A...,5.0,Positive
4,on 01/03/16 18:03 PM (PST),One owner,2006 Ford Mustang Coupe GT Premium 2dr Coupe (...,One owner,I bought this car spankin new and i still am ...,5.0,Positive


In [23]:
#Converting DataFrame to JSON-like format
result = df.to_dict(orient="records")  # Convert rows to list of dictionaries
formatted_json = json.dumps(result[0], indent=2)  # Convert the first record to JSON with indentation

#Printing Final Output in the formatted JSON
print(formatted_json)

{
  "Review_Date": " on 06/06/18 14:19 PM (PDT)",
  "Author_Name": "Vicki ",
  "Vehicle_Title": "2006 Ford Mustang Coupe GT Premium 2dr Coupe (4.6L 8cyl 5M)",
  "Review_Title": "2006 Mustang GT",
  "Review": " Doesn\u2019t disappoint",
  "Rating": 5.0,
  "Sentiment": "Positive"
}
